1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

### 1. 문서의 내용을 읽는다 & 2. 문서를 쪼갠다.

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = Docx2txtLoader('./tax.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

187

### 3. 임베딩 -> 벡터 데이터베이스에 저장

In [4]:
from langchain_chroma import Chroma
# from langchain_naver.embeddings import ClovaXEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings


# embeddings = ClovaXEmbeddings(model="bge-m3")
embeddings = OllamaEmbeddings(model="nomic-embed-text")
database = Chroma.from_documents(documents=document_list, embedding=embeddings, collection_name="chroma-tax", persist_directory='./.chroma')

### 4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색

In [5]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)
retrieved_docs

[Document(id='b3fc5364-57fc-4a22-8e02-786403719e44', metadata={'source': './tax.docx'}, page_content='다. 외국법인이 발행하였거나 외국에 있는 시장에 상장된 주식등으로서 대통령령으로 정하는 것\n\n4. 다음 각 목의 어느 하나에 해당하는 자산(이하 이 장에서 “기타자산”이라 한다)의 양도로 발생하는 소득\n\n가. 사업에 사용하는 제1호 및 제2호의 자산과 함께 양도하는 영업권(영업권을 별도로 평가하지 아니하였으나 사회통념상 자산에 포함되어 함께 양도된 것으로 인정되는 영업권과 행정관청으로부터 인가ㆍ허가ㆍ면허 등을 받음으로써 얻는 경제적 이익을 포함한다)\n\n나. 이용권ㆍ회원권, 그 밖에 그 명칭과 관계없이 시설물을 배타적으로 이용하거나 일반이용자보다 유리한 조건으로 이용할 수 있도록 약정한 단체의 구성원이 된 자에게 부여되는 시설물 이용권(법인의 주식등을 소유하는 것만으로 시설물을 배타적으로 이용하거나 일반이용자보다 유리한 조건으로 시설물 이용권을 부여받게 되는 경우 그 주식등을 포함한다)\n\n다. 법인의 자산총액 중 다음의 합계액이 차지하는 비율이 100분의 50 이상인 법인의 과점주주(소유 주식등의 비율을 고려하여 대통령령으로 정하는 주주를 말하며, 이하 이 장에서 “과점주주”라 한다)가 그 법인의 주식등의 100분의 50 이상을 해당 과점주주 외의 자에게 양도하는 경우(과점주주가 다른 과점주주에게 양도한 후 양수한 과점주주가 과점주주 외의 자에게 다시 양도하는 경우로서 대통령령으로 정하는 경우를 포함한다)에 해당 주식등\n\n\u3000\u3000\u30001) 제1호 및 제2호에 따른 자산(이하 이 조에서 “부동산등”이라 한다)의 가액\n\n\u3000\u3000\u30002) 해당 법인이 직접 또는 간접으로 보유한 다른 법인의 주식가액에 그 다른 법인의 부동산등 보유비율을 곱하여 산출한 가액. 이 경우 다른 법인의 범위 및 부동산등 보유비율의 계산방법 등은 대통령령

### 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [6]:
from langchain_naver.chat_models import ChatClovaX


prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

# llm = ChatOllama(model="llama3.2:1b")
llm = ChatClovaX(model_name="HCX-005-BETA")
ai_message = llm.invoke(prompt)

ai_message.content


'연봉 5천만 원인 직장인의 소득세를 구하기 위해서는 다음과 같은 단계를 거쳐야 합니다.\n\n1. **기본공제**: 본인과 배우자 그리고 부양가족 1명당 150만 원씩 공제됩니다.\n2. **근로소득공제**: 총급여액에 따라 최소 500만 원부터 최대 1천 500만 원까지 공제됩니다.\n3. **과세표준**: 위에서 공제된 금액들을 제외한 값입니다.\n4. **세율 적용**: 과세표준 금액에 따라 6%부터 최대 45%까지의 세율이 적용됩니다.\n5. **누진공제**: 각 구간별 세액에서 누진공제액이 차감됩니다.\n6. **지방소득세 추가**: 최종 결정세액에 지방소득세가 10% 추가로 부과됩니다.\n\n이를 토대로 아래와 같이 계산할 수 있습니다.\n\n* 기본공제: 본인 150만원 + 배우자 150만원 + 자녀 150만원 = 450만원\n* 근로소득공제: 500만원\n* 과세표준: 5,000 - (450+500) = 4,050만원\n* 세율적용: 1200만원이하 6%, 1200~4600만원이하 15%\n* 소득세: 126(600 x 0.06) + 585(3900 x 0.15)=711만원\n* 지방소득세: 71.1(711 x 0.1)\n* 총 부담세액: 782.1만원\n\n따라서 위의 과정을 거친 결과, 연봉 5천만 원인 직장인이 내야 하는 소득세와 지방소득세의 총액은 대략 **782만 1천 원** 정도 될 것으로 예상됩니다. 하지만 이는 매우 일반적인 경우에 해당하므로 개인의 상황에 따라 달라질 수 있으니 자세한 사항은 국세청 홈페이지나 세무사 상담을 통해 확인하시는 것을 추천 드립니다.'

## langchain hub 를 이용해서 RAG 구성

 1. rag prompt 가져오기

In [1]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

/Users/user/Git/til/python/inflearn-llm-application/.venv/lib/python3.10/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

2. 이미 저장된 내용의 chroma db 가져오기

In [11]:
from langchain_chroma import Chroma
from langchain_ollama.embeddings import OllamaEmbeddings


embeddings = OllamaEmbeddings(model="nomic-embed-text")
database = Chroma(collection_name="chroma-tax", persist_directory='./.chroma', embedding_function=embeddings,)

3. RetrievalQA 로 db 와 llm chain

In [12]:
from langchain.chains import RetrievalQA
from langchain_naver.chat_models import ChatClovaX

llm = ChatClovaX(model_name="HCX-005-BETA")
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)


4. chain 에서 질의하기

In [14]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
ai_message = qa_chain({"query": query})
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만 원인 직장인의 소득세를 정확히 알기 위해서는 공제 항목과 세율 적용 등에 따라 달라지기 때문에, 국세청 홈페이지나 세금 관련 사이트에서 제공하는 간이세액표를 참고하시거나, 전문가와의 상담을 통해 정확한 금액을 확인하시는 것이 좋습니다. 일반적으로 근로소득공제, 기본공제, 인적공제 등의 다양한 공제 항목을 차감한 뒤 구간별 종합소득세율이 적용되어 최종적인 세액이 결정됩니다. 따라서 개인의 상황에 따라 실제 부담하는 소득세가 다를 수 있습니다.'}